In [ ]:
import polars as pl
import heapq

def build_long_cross_sections(symbol_dfs: dict[str, pl.DataFrame]) -> pl.DataFrame:
    # 准备事件堆
    events = []
    for symbol, df in symbol_dfs.items():
        df_sorted = df.sort("timestamp")
        for ts in df_sorted["timestamp"].to_list():
            events.append((ts, symbol))
    heapq.heapify(events)

    latest = {}      # symbol -> 最新行（dict）
    snapshots = []   # 存储 long format 的每条记录

    while events:
        ts, symbol = heapq.heappop(events)
        df = symbol_dfs[symbol]
        # 找出 <= ts 的最近一行
        row = (
            df
            .filter(pl.col("timestamp") <= ts)
            .sort("timestamp", descending=True)
            .head(1)
            .to_dicts()[0]
        )
        latest[symbol] = row

        # 如果所有币种都已经有数据，生成一组 snapshot（多行）
        if len(latest) == len(symbol_dfs):
            for sym, rec in latest.items():
                row_data = {"timestamp": ts, "symbol": sym}
                for k, v in rec.items():
                    if k != "timestamp":
                        row_data[k] = v
                snapshots.append(row_data)

    return pl.DataFrame(snapshots)
